In [5]:
import pandas as pd
d = pd.read_excel("C:\\Users\\HP\\Desktop\\PES_SEM_8\\KGraphs-QA\\qa\\QA.xlsx", sheetname='Question-Intent')

C:\Users\HP\Anaconda3\lib\site-packages\pandas\io\excel.py:329: FutureWarning: The `sheetname` keyword is deprecated, use `sheet_name` instead
  **kwds)


In [6]:
from nltk import word_tokenize, pos_tag
from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from nltk.stem.wordnet import WordNetLemmatizer
def penn_to_wn(tag):
    """ Convert between a Penn Treebank tag to a simplified Wordnet tag """
    if tag.startswith('N'):
        return 'n'
 
    if tag.startswith('V'):
        return 'v'
 
    if tag.startswith('J'):
        return 'a'
 
    if tag.startswith('R'):
        return 'r'
 
    return None
 
def tagged_to_synset(word, tag):
    wn_tag = penn_to_wn(tag)
    if wn_tag is None:
        return None
 
    try:
        return wn.synsets(word, wn_tag)[0]
    except:
        return None
def check_similarity(sentence1, sentence2):
    """ compute the sentence similarity using Wordnet """
    # Tokenize and tag
    sentence1 = pos_tag(word_tokenize(sentence1))
    sentence2 = pos_tag(word_tokenize(sentence2))
 
    # Get the synsets for the tagged words
    synsets1 = [tagged_to_synset(*tagged_word) for tagged_word in sentence1]
    synsets2 = [tagged_to_synset(*tagged_word) for tagged_word in sentence2]
 
    # Filter out the Nones
    synsets1 = [ss for ss in synsets1 if ss]
    synsets2 = [ss for ss in synsets2 if ss]
 
    score, count = 0.0, 0
    #print(synsets1)
    #print(synsets2)
       # For each word in the first sentence
    for synset in synsets1:
        # Get the similarity value of the most similar word in the other sentence
        best_score = list([synset.path_similarity(ss) for ss in synsets2])
        best_score= list(filter(lambda a: a != None, best_score))
        if(best_score==[]):
            best_score =0
        else:
            best_score = max(best_score)
        # Check that the similarity could have been computed
        if best_score is not None:
            score += best_score
            count += 1
 
    # Average the values
    if (count!= 0):
        score /= count
    return score

In [7]:
def sortThird(val): 
    return val[2]  
  
def get_Intent(q):
    all=[]
    for x in list(d["Question"]):
        print(x  , "||" , list(d[d["Question"] == x]["Intent"])[0], "||", check_similarity(q, x))
        l=[]
        l.append(x)
        l.append(list(d[d["Question"] == x]["Intent"])[0])
        l.append(check_similarity(q, x))
        all.append(l)
    all.sort(key = sortThird, reverse=True)
    return all[0][1]

In [9]:
get_Intent("what do I take for problem")

what are symptoms of Problem || disease_symptom || 0.5277777777777778
what is problem caused by disease? || symptom_disease || 0.5277777777777778
what are the side effects of treatment || treatment_side_effects || 0.22474747474747472
What medications are best for the treatment of problem? || disease_treatment || 0.5277777777777778
what diseases are cured by treatment || treatment_disease || 0.22008547008547005
 what problem is a test conducted for || test_problem || 0.5277777777777778
what tests are conducted for Problem || problem_test || 0.5277777777777778
What are the signs that I have Problem? || disease_symptom || 0.22474747474747472
is problem a symptom of problem || symptom_disease || 0.5277777777777778
what happens when I take treatment || treatment_side_effects || 0.47008547008547
I have problem, what treatment do I take? || disease_treatment || 1.0
is treatment helpful for problem?  || treatment_disease || 0.5277777777777778
what are the indications that I have problem? || di

'disease_treatment'